# Project Intermediate Report - Mancala (Random vs. Random)
By: Luca Chaia and Adara Noble

## Overview
- A working mancala engine created from the `Mancala` class from HW #6
- A light simulation system that runs **random vs. random** for `N` games, accumulates statistics and displays the results
- Deterministic runs supported via a seed setter

In [1]:
import sys
#sys.builtin_module_names
!{sys.executable} -m pip install numpy
#https://stackoverflow.com/questions/48754352/python-package-not-found-in-jupyter-even-after-running-pip-install?rq=3

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 30.2 MB/s eta 0:00:0031m34.7 MB/s eta 0:00:01


In [2]:
import random
import numpy as np

## The Mancala Engine

In [48]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4, quiet=False):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.current_player = 1 # start with player 1
        self.moves = [] 
        self.p1_pits_index = [0, self.pits_per_player-1] 
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1] 
        self.p2_mancala_index = len(self.board)-1
        self.quiet = quiet
        self._print = print if not quiet else (lambda *a, **k: None)
        self._rand = random
        
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

        self.winner = 0 # stores the winner for easy access later

    
    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)

    
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        pit -= 1 #accounts for 1-indexing

        #populates valid_range with the indices of pits belonging to the current_player
        if self.current_player == 1:
            valid_range = range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)
        else:
            valid_range = range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)

        #checks whether the selected pit is within the range of pits belonging to the current_player
        if pit < len(valid_range):
            #creates a list with all the valid ranges (converted from range()) then grabs the value at index 'pit'
            # aka: converts the pit number to match the pit's index in the board
            pit_index = list(valid_range)[pit]
        else: 
            print("Issue is with range")
            return False
        
        if self.board[pit_index] == 0: #checks if the pit is empty at the (determined to be valid) index
            print("Problem is with stones")
            return False

        return True

    
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        valid_pits = []
        if self.current_player == 1:
            start, end = self.p1_pits_index #unpacks p1's pit indices
        else:
            start, end = self.p2_pits_index #unpacks p2's pit indices

        for i in range(start, end + 1): #generates indices of current player's pits
            if self.board[i] > 0: #checks whether each pit is empty
                valid_pits.append(i - start + 1) #if not empty, pit index gets added to the list of valid indices

        if not valid_pits: #if this happens, there is an issue with the win calculation or with this generator
            return None
        
        return self._rand.choice(valid_pits) #randomly chooses a pit (move) from the list of valid moves

    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        if not self.valid_move(pit): #checks if the move is valid 
            print(f"INVALID MOVE: Pit {pit} by Player {self.current_player}")
            return self.board

        if self.winning_eval(): #checks to see if the game has already been won
            print("GAME OVER")
            return self.board

        pit -= 1 #accounts for 1-indexing

        #converts the pit number to match the pit's index in the board
        if self.current_player == 1:
            pit_index = self.p1_pits_index[0] + pit
        else:
            pit_index = self.p2_pits_index[0] + pit

        stones = self.board[pit_index] #grabs the number of stones in the pit 
        self.board[pit_index] = 0 #empties the pit
        index = pit_index #initializes the iterable

        while stones > 0:
            index = (index + 1) % len(self.board) #if we're past the bounds of the board (which translates
            # to index equaling the lenth of the board), our index is set to the start of the list

            #ensures that you don't place stones in the other player's mancala
            if self.current_player == 1 and index == self.p2_mancala_index:
                continue
            if self.current_player == 2 and index == self.p1_mancala_index:
                continue

            self.board[index] += 1 #adds a stone to the current pit
            stones -= 1 #removes a stone from our 'hand'

        #implements the 'if a player's last stone lands in an empty pit on their side of the board, take that stone
        # and all the stones in the opposite pit (on the other player's side) and put them into your own mancala' rule
        if self.current_player == 1:
            #checks that the final pit is on player 1's side, and that the pit was empty (and now has 1 stone)
            if index in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1) and self.board[index] == 1:
                opposite_index = self.p2_pits_index[1] - (index - self.p1_pits_index[0])
                #calculates P2's index
                captured = self.board[opposite_index]
                #places the stones into P1's mancala
                self.board[self.p1_mancala_index] += captured + 1
                self.board[index] = 0
                self.board[opposite_index] = 0
        else:
            #checks that the final pit is on player 2's side, and that the pit was empty (and now has 1 stone)
            if index in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1) and self.board[index] == 1:
                opposite_index = self.p1_pits_index[1] - (index - self.p2_pits_index[0])
                #calculates P1's index
                captured = self.board[opposite_index]
                #places the stones into P2's mancala
                self.board[self.p2_mancala_index] += captured + 1
                self.board[index] = 0
                self.board[opposite_index] = 0

        #adds the move to 'history'
        self.moves.append((self.current_player, pit + 1))

        #implements the 'if your final stone was dropped into your mancala, take another turn, otherwise 
        # switch players' rule
        #if (self.current_player == 1 and index != self.p1_mancala_index) or (self.current_player == 2 and index != self.p2_mancala_index):
            #self.current_player = 2 if self.current_player == 1 else 1

        #skips the previously established rule, swaps players every turn
        self.current_player = 2 if self.current_player == 1 else 1
        
        return self.board

    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state (if either of the players' pits are all empty)
        """
        #checks each pit in each players' side of the board to see if they're empty
        p1_side_empty = all(stone == 0 for stone in self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1])
        p2_side_empty = all(stone == 0 for stone in self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1])

        
        if p1_side_empty or p2_side_empty: # win condition met
            #adds the sum of all stones left within each players' pit to their respective mancalas
            self.board[self.p1_mancala_index] += sum(self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1])
            self.board[self.p2_mancala_index] += sum(self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1])
            #empties each player's pits
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                self.board[i] = 0
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                self.board[i] = 0

            #checks who won (has more stones in mancala)
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                self.winner = 1 # winner is player 1
                self._print("Player 1 wins!")
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                self.winner = 2 # winner is player 2
                self._print("Player 2 wins!")
            else:
                #print("It’s a tie!")
                pass
            return True
        return False

## The AI

In [56]:
utilityEQ = {
    "Mancala Stones":
     "max(board[self.game.p1_mancala_index], board[self.game.p2_mancala_index]) - min(board[self.game.p1_mancala_index], board[self.game.p2_mancala_index])",
    "Test":
     "1",
            }
class AI:
    def __init__(self, game, plies, utility = "Mancala Stones"):
        self.game = game #this is so we don't have to keep adding and removing variables if later utilities require different things
        self.plies = plies
        self.utility = utilityEQ.get(utility)
        self.player1 = False 

    def AIMove(self, board):
        #converts the pit to match what the Mancala engine expects
        _, pit = self.evalMax(board[:]) 
        return (pit - self.game.p2_pits_index[0] + 1)

    def evalMin(self, board, currPly = 0):
        if currPly == self.plies or self.winning_eval(board):
            return (eval(self.utility), None)

        currPly += 1
        moves = []
        
        for pit in range( self.game.p1_pits_index[0], self.game.p1_pits_index[1] + 1 ):
            if board[pit] > 0:
                newBoard = self.play(board[:], pit)
                moves.append((self.evalMax(newBoard, currPly)[0], pit))

        #min and max don't work with lists of tuples :(
        m = (np.inf, np.inf)
        for i in range( len(moves) ):
            val, ind = moves[i]
            if val < m[0]:
                m = (val, ind)

        return m

    def evalMax(self, board, currPly = 0):
        if currPly == self.plies or self.winning_eval(board):
            return (eval(self.utility), None)

        currPly += 1
        moves = []
        
        for pit in range( self.game.p2_pits_index[0], self.game.p2_pits_index[1] + 1 ):
            if board[pit] > 0:
                newBoard = self.play(board[:], pit)
                moves.append((self.evalMin(newBoard, currPly)[0], pit))

        #min and max don't work with lists of tuples :(
        m = (-1,-1)
        for i in range( len(moves) ):
            val, ind = moves[i]
            if val > m[0]:
                m = (val, ind)
        
        return m

    def winning_eval(self, board):
        p1_side_empty = all(stone == 0 for stone in board[self.game.p1_pits_index[0]:self.game.p1_pits_index[1] + 1])
        p2_side_empty = all(stone == 0 for stone in board[self.game.p2_pits_index[0]:self.game.p2_pits_index[1] + 1])
        if p1_side_empty or p2_side_empty: 
            return True
        return False

    def play(self, board, pit_index): 
        #determines the 'current player' based on the pit
        current_player = 1 if pit_index in range(self.game.p1_pits_index[0],self.game.p1_mancala_index + 1) else 2
        
        #I think this should implement the mancala rule?
        #round2 = False
        #if (current_player==1 and (pit_index+board[pit])%len(board)==self.game.p1_mancala_index) or {
        #    current_player==2 and (pit_index+board[pit])%len(board)==self.game.p1_mancala_index}:
        #    round2 = True
        
        #a mostly exact copy of the play function from the game class
        stones = board[pit_index]
        board[pit_index] = 0 
        index = pit_index
        while stones > 0:
            index = (index + 1) % len(board)
            if (current_player == 1 and index == self.game.p2_mancala_index) or (current_player == 2 and index == self.game.p1_mancala_index):
                continue
            board[index] += 1
            stones -= 1
        if current_player == 1:
            if index in range(self.game.p1_pits_index[0], self.game.p1_pits_index[1] + 1) and board[index] == 1:
                opposite_index = self.game.p2_pits_index[1] - (index - self.game.p1_pits_index[0])
                captured = board[opposite_index]
                board[self.game.p1_mancala_index] += captured + 1
                board[index] = 0
                board[opposite_index] = 0
        else:
            if index in range(self.game.p2_pits_index[0], self.game.p2_pits_index[1] + 1) and board[index] == 1:
                opposite_index = self.game.p1_pits_index[1] - (index - self.game.p2_pits_index[0])
                captured = board[opposite_index]
                board[self.game.p2_mancala_index] += captured + 1
                board[index] = 0
                board[opposite_index] = 0
        
        #potential mancala rule continued
        #if round2 and current_player == 1:
        #    self.play(board, self.game.p1_mancala_index)
        #elif round2 and current_player == 2:
        #    self.play(board, self.game.p1_mancala_index)

        return board



In [61]:
testGame = Mancala()
testAI = AI(testGame, 3)
while not testGame.winning_eval():
    pit = testGame.random_move_generator()
    if pit is None:
        testGame._print("Error")
        break
    testGame.play(pit)
    
    #new
    if not testGame.winning_eval():
        pit = testAI.AIMove(testGame.board[:])
        testGame.play(pit)
    else:
        break

Player 2 wins!


## Data structures used for statistics
### `PlayerStats`
Tracks results for one player (`p1` or `p2`)

In [5]:
from dataclasses import dataclass, field

@dataclass
class PlayerStats:
    wins: int = 0
    losses: int = 0
    ties: int = 0

### `MatchStats`
Holds both players and global counters

In [6]:
@dataclass
class MatchStats:
    p1: PlayerStats = field(default_factory=PlayerStats)
    p2: PlayerStats = field(default_factory=PlayerStats)
    turns_total: int = 0
    games: int = 0

### Simulation runner: `run_matches(...)`
**How it works:**
- Creates a fresh `Mancala` game each match with `quiet=True` by default (so the engine doesn’t spam prints during tests).

- Generates random valid moves for the current player until `winning_eval()` declares the game over.

- Counts turns by incrementing once per `play(...)`.

- Updates separate stats for Player 1 and Player 2 (wins, losses, ties).

Returns a small result bundle with:

- `stats` (typed counters),

- `avg_turns` (mean turns per game),

- `turns_per_game` (raw list in case we want to do further data analysis like a histogram)

**Determinism:** If seed is provided, Python’s random is seeded so repeated runs are reproducible.

**Quiet mode:** Passing `quiet=True` silences engine prints (via `self._print` inside ``Mancala).

In [20]:
def run_matches(num_games=100, *, pits_per_player=6, stones_per_pit=4, quiet=True, seed=None):
    import numpy as np
    import random

    if seed is not None:
        random.seed(seed)

    stats = MatchStats()
    turns_per_game = []

    for _ in range(num_games):
        game = Mancala(pits_per_player=pits_per_player, stones_per_pit=stones_per_pit, quiet=quiet)
        #new
        ai = AI(game, 3)
        #
        turns = 0

        while not game.winning_eval():
            pit = game.random_move_generator()
            if pit is None:
                game._print("Error")
                break
            game.play(pit)
            turns += 1

            #new
            if not game.winning_eval():
                pit = ai.AIMove(game.board[:])
                game.play(pit)
            else:
                break
            #

        # update stats
        if game.winner == 1:
            stats.p1.wins += 1
            stats.p2.losses += 1
        elif game.winner == 2:
            stats.p2.wins += 1
            stats.p1.losses += 1
        else:
            stats.p1.ties += 1
            stats.p2.ties += 1

        stats.turns_total += turns
        stats.games += 1
        turns_per_game.append(turns)

    # result bundle
    return {
        "stats": stats,
        "avg_turns": float(np.mean(turns_per_game)) if turns_per_game else 0.0,
        "turns_per_game": turns_per_game,
    }

### Report printing helper: `print_stats(result)`
- Prints **win/loss/tie** pct per player
- First-move advantage as the difference `P1 win% − P2 win%` (positive = P1 favored).

In [8]:
def print_stats(result):
    s = result["stats"]
    n = s.games or 1  # avoid div by zero
    p1_win_pct = 100 * s.p1.wins / n
    p2_win_pct = 100 * s.p2.wins / n
    tie_pct   = 100 * s.p1.ties / n  # same as p2.ties
    avg_turns = result["avg_turns"]

    print(f"Player 1 — Won: {s.p1.wins} ({p1_win_pct:.1f}%), "
          f"Lost: {s.p1.losses} ({100 * s.p1.losses / n:.1f}%), "
          f"Tied: {s.p1.ties} ({tie_pct:.1f}%)")

    print(f"Player 2 — Won: {s.p2.wins} ({p2_win_pct:.1f}%), "
          f"Lost: {s.p2.losses} ({100 * s.p2.losses / n:.1f}%), "
          f"Tied: {s.p2.ties} ({tie_pct:.1f}%)")

    print(f"Average turns per game: {avg_turns:.2f}")

    advantage = p1_win_pct - p2_win_pct # “First-move advantage” (positive = P1 favored)
    print(f"First-move advantage (P1 win% − P2 win%): {advantage:+.1f} percentage points")


## Test results

In [49]:
res = run_matches(100, quiet=True, seed=42)
print_stats(res)

Problem is with stones
INVALID MOVE: Pit 4 by Player 2
Problem is with stones
INVALID MOVE: Pit 3 by Player 1
Problem is with stones
INVALID MOVE: Pit 2 by Player 2
Problem is with stones
INVALID MOVE: Pit 2 by Player 1
Problem is with stones
INVALID MOVE: Pit 2 by Player 2
Problem is with stones
INVALID MOVE: Pit 3 by Player 1
Problem is with stones
INVALID MOVE: Pit 3 by Player 2
Problem is with stones
INVALID MOVE: Pit 2 by Player 2
Problem is with stones
INVALID MOVE: Pit -1 by Player 1
Problem is with stones
INVALID MOVE: Pit 4 by Player 2
Problem is with stones
INVALID MOVE: Pit 2 by Player 1
Problem is with stones
INVALID MOVE: Pit 2 by Player 2
Problem is with stones
INVALID MOVE: Pit 4 by Player 1
Problem is with stones
INVALID MOVE: Pit 1 by Player 2
Problem is with stones
INVALID MOVE: Pit 1 by Player 1
Problem is with stones
INVALID MOVE: Pit 1 by Player 2
Problem is with stones
INVALID MOVE: Pit 1 by Player 1
Problem is with stones
INVALID MOVE: Pit 1 by Player 2
Problem i

By running our simulation over a large amount of games, we do see a slight **first-move advantage**. While we rarely see ties, there are cases where it happens